# DEX on zkSync Era - dataset parser

- Krzysztof Gogol
- email: [gogol@ifi.uzh.ch](mailto:gogol@ifi.uzh.ch)
- Website: https://gogol.substack.com/

In [3]:
import gzip
import pickle
import os
from tqdm.notebook import tqdm
import pandas as pd
#from web3 import Web3
import web3
import numpy as np
import matplotlib.pyplot as plt



/Users/kris/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
path_dir = './data/'
os.makedirs(path_dir, exist_ok=True)

In [5]:
file1 = 'SyncSwap-USDC-ETH_Swap.csv.gz'
swap_df = pd.read_csv(path_dir+'/'+file1,
                        sep=',', compression='gzip')
swap_df.head()

file2 = 'SyncSwap-USDC-ETH_Sync.csv.gz'
sync_df = pd.read_csv(path_dir+'/'+file2,
                        sep=',', compression='gzip')
sync_df.head()


,Unnamed: 0,blockNumber,transactionHash,address,transactionIndex,logIndex,topics_0,topics_1,topics_2,topics_3,data,timestamp,event_name
0,0,13106,0x7bc13feafc02db763e5db405de99d69364b0979763af...,0x80115c708e12edd42e504c1cd52aea96c547c05c,7,36,0xcf2aa50876cdfbb541206f89af0ee78d44a2abf8d328...,NaN,NaN,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:23:49,Sync
1,1,13463,0xe796d825e6a33a55483d273b101cd567c6e57caf39b5...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,6,0xcf2aa50876cdfbb541206f89af0ee78d44a2abf8d328...,NaN,NaN,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:31:39,Sync
2,2,13849,0xe0590193ade2eaaedb063e270e62afbdb5bcd54eb0a3...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,6,0xcf2aa50876cdfbb541206f89af0ee78d44a2abf8d328...,NaN,NaN,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:39:58,Sync
3,3,13859,0x070a3c311dab420e0bb41ebee9c360ee49f8dcae7fc4...,0x80115c708e12edd42e504c1cd52aea96c547c05c,2,12,0xcf2aa50876cdfbb541206f89af0ee78d44a2abf8d328...,NaN,NaN,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:40:09,Sync
4,4,13911,0xa23d767e5959ae44441eef7d979743b38e7b1154937c...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,9,0xcf2aa50876cdfbb541206f89af0ee78d44a2abf8d328...,NaN,NaN,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:41:08,Sync


In [6]:
sync_df['transactionHash'][3]

'0x070a3c311dab420e0bb41ebee9c360ee49f8dcae7fc4e208d21ef3245ba994b4'

In [7]:
file_market_data = 'MarketData/ETH-USD.csv'
prices_df = pd.read_csv(file_market_data)
prices_df['timestamp'] = pd.to_datetime(prices_df['Date'])
prices_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,timestamp
0,2023-02-01,1586.488159,1644.727539,1566.857300,1641.792725,1641.792725,8116969489,2023-02-01
1,2023-02-02,1641.365967,1704.458130,1641.322632,1643.241577,1643.241577,10558081069,2023-02-02
2,2023-02-03,1642.904663,1670.696899,1634.223389,1664.745605,1664.745605,8169519805,2023-02-03
3,2023-02-04,1664.472290,1690.099609,1648.189209,1667.059204,1667.059204,5843302512,2023-02-04
4,2023-02-05,1667.166504,1671.770264,1616.391846,1631.645874,1631.645874,6926696531,2023-02-05


In [8]:
#SWAP_FUNCTION_HASH = web3.Web3.keccak(text="Swap(address,uint256,uint256,uint256,uint256,address)").hex()
# Swap (index_topic_1 address sender, uint256 amount0In, uint256 amount1In, uint256 amount0Out, uint256 amount1Out, index_topic_2 address to)

def convertAmount1(dataString):
    return int(dataString[2:66], 16)

CONST_ETH = 1e18
CONST_USDC = 1e6

swap_df['amount0In'] = np.full(len(swap_df),0)
swap_df['amount1In'] = np.full(len(swap_df),0)
swap_df['amount0Out'] = np.full(len(swap_df),0)
swap_df['amount1Out'] = np.full(len(swap_df),0)


swap_df['amount0In'] = swap_df.apply(lambda row: int(row['data'][2:66], 16), axis=1) / CONST_USDC
swap_df['amount1In'] = swap_df.apply(lambda row: int(row['data'][66:130], 16), axis=1) / CONST_ETH
swap_df['amount0In'] = swap_df.apply(lambda row: int(row['data'][130:194], 16), axis=1) / CONST_USDC
swap_df['amount0Out'] = swap_df.apply(lambda row: int(row['data'][194:258], 16), axis=1) / CONST_ETH



In [10]:
swap_df['price_buy_ETH'] = np.full(len(swap_df),0)


#Spot Price BTC/ETH
#swap_df['price_buy_ETH'] = swap_df['amount0In'] / swap_df['amount1In']
#swap_df['price_buy_USDC'] = swap_df['amount0Out'] / swap_df['amount1Out']

swap_df.head()


,Unnamed: 0,blockNumber,transactionHash,address,transactionIndex,logIndex,topics_0,topics_1,topics_2,topics_3,data,timestamp,event_name,amount0In,amount1In,amount0Out,amount1Out,price_buy_ETH
0,0,13463,0xe796d825e6a33a55483d273b101cd567c6e57caf39b5...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,4,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x00000000000000000000000073459ef0a33ea718211e...,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:31:39,Swap,1.313849,0.001,0.0,0,0
1,1,13849,0xe0590193ade2eaaedb063e270e62afbdb5bcd54eb0a3...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,4,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000e4d9f174e9b9f9878360...,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:39:58,Swap,0.094332,0.0001,0.0,0,0
2,2,13859,0x070a3c311dab420e0bb41ebee9c360ee49f8dcae7fc4...,0x80115c708e12edd42e504c1cd52aea96c547c05c,2,10,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000e4d9f174e9b9f9878360...,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:40:09,Swap,0.089620,0.0001,0.0,0,0
3,3,14099,0xc8b9f63c526b90d2a835481dbc9003197276db4b627e...,0x80115c708e12edd42e504c1cd52aea96c547c05c,1,7,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000e776b14985b0ea61d298...,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:45:01,Swap,0.000873,0.000001,0.0,0,0
4,4,14549,0xfe651bc486680777278b99774247356618c32d0027f5...,0x80115c708e12edd42e504c1cd52aea96c547c05c,1,7,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x0000000000000000000000007b22f4e0b10b2d3a09f8...,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:54:13,Swap,2.184087,0.002563,0.0,0,0


In [11]:

sync_df['reserve_0'] = np.full(len(sync_df),0) #Amount of ETH in the pool
sync_df['reserve_1'] = np.full(len(sync_df),0) #Amount of BTC in the pool

sync_df['reserve_0'] = sync_df.apply(lambda row: int(row['data'][2:66], 16), axis=1) / CONST_USDC
sync_df['reserve_1'] = sync_df.apply(lambda row: int(row['data'][66:130], 16), axis=1) / CONST_ETH

#Spot Price BTC/ETH
sync_df['spot_price'] = sync_df['reserve_0'] / sync_df['reserve_1']
sync_df.head()

,Unnamed: 0,blockNumber,transactionHash,address,transactionIndex,logIndex,topics_0,topics_1,topics_2,topics_3,data,timestamp,event_name,reserve_0,reserve_1,spot_price
0,0,13106,0x7bc13feafc02db763e5db405de99d69364b0979763af...,0x80115c708e12edd42e504c1cd52aea96c547c05c,7,36,0xcf2aa50876cdfbb541206f89af0ee78d44a2abf8d328...,NaN,NaN,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:23:49,Sync,5.000000,0.0028,1785.714286
1,1,13463,0xe796d825e6a33a55483d273b101cd567c6e57caf39b5...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,6,0xcf2aa50876cdfbb541206f89af0ee78d44a2abf8d328...,NaN,NaN,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:31:39,Sync,3.686151,0.0038,970.039737
2,2,13849,0xe0590193ade2eaaedb063e270e62afbdb5bcd54eb0a3...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,6,0xcf2aa50876cdfbb541206f89af0ee78d44a2abf8d328...,NaN,NaN,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:39:58,Sync,3.591819,0.0039,920.979231
3,3,13859,0x070a3c311dab420e0bb41ebee9c360ee49f8dcae7fc4...,0x80115c708e12edd42e504c1cd52aea96c547c05c,2,12,0xcf2aa50876cdfbb541206f89af0ee78d44a2abf8d328...,NaN,NaN,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:40:09,Sync,3.502199,0.004,875.54975
4,4,13911,0xa23d767e5959ae44441eef7d979743b38e7b1154937c...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,9,0xcf2aa50876cdfbb541206f89af0ee78d44a2abf8d328...,NaN,NaN,NaN,0x00000000000000000000000000000000000000000000...,2023-03-24 14:41:08,Sync,3.686151,0.00421,875.54975


In [12]:
swap_df['last_price'] = swap_df['price_buy_ETH']
#for j, row in swap_df.iterrows():
#    if np.isnan(swap_df['last_price'].iloc[j]) :
#        swap_df['last_price'].iloc[j]  = swap_df['price_buy_BTC'].iloc[j]

#join swap_df and sync_df
swap_df = swap_df.merge(sync_df[['blockNumber','reserve_0', 'reserve_1', 'spot_price']], on='blockNumber', how='left')
#swap_df['diff'] = swap_df['last_price'] - swap_df['spot_price']
swap_df.head()


,Unnamed: 0,blockNumber,transactionHash,address,transactionIndex,logIndex,topics_0,topics_1,topics_2,topics_3,...,event_name,amount0In,amount1In,amount0Out,amount1Out,price_buy_ETH,last_price,reserve_0,reserve_1,spot_price
0,0,13463,0xe796d825e6a33a55483d273b101cd567c6e57caf39b5...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,4,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x00000000000000000000000073459ef0a33ea718211e...,NaN,...,Swap,1.313849,0.001,0.0,0,0,0,3.686151,0.0038,970.039737
1,1,13849,0xe0590193ade2eaaedb063e270e62afbdb5bcd54eb0a3...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,4,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000e4d9f174e9b9f9878360...,NaN,...,Swap,0.094332,0.0001,0.0,0,0,0,3.591819,0.0039,920.979231
2,2,13859,0x070a3c311dab420e0bb41ebee9c360ee49f8dcae7fc4...,0x80115c708e12edd42e504c1cd52aea96c547c05c,2,10,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000e4d9f174e9b9f9878360...,NaN,...,Swap,0.089620,0.0001,0.0,0,0,0,3.502199,0.004,875.54975
3,3,14099,0xc8b9f63c526b90d2a835481dbc9003197276db4b627e...,0x80115c708e12edd42e504c1cd52aea96c547c05c,1,7,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000e776b14985b0ea61d298...,NaN,...,Swap,0.000873,0.000001,0.0,0,0,0,3.685278,0.004211,875.134526
4,4,14549,0xfe651bc486680777278b99774247356618c32d0027f5...,0x80115c708e12edd42e504c1cd52aea96c547c05c,1,7,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x0000000000000000000000007b22f4e0b10b2d3a09f8...,NaN,...,Swap,2.184087,0.002563,0.0,0,0,0,87.385267,0.104911,832.94317


In [13]:
swap_df['timestamp'] = pd.to_datetime(swap_df['timestamp'])
swap_df.set_index('timestamp', inplace=True)
df_per_day = swap_df.resample('D').last()

df_per_day.head()

,Unnamed: 0,blockNumber,transactionHash,address,transactionIndex,logIndex,topics_0,topics_1,topics_2,topics_3,...,event_name,amount0In,amount1In,amount0Out,amount1Out,price_buy_ETH,last_price,reserve_0,reserve_1,spot_price
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-03-24,1204,43554,0xad594851f951591e5b36f92f35db9b8ef602edb207d2...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,4,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x0000000000000000000000003545744fd133dddfb26b...,NaN,...,Swap,40.900104,0.025069,0.0,0,0,0,1.387058e+04,8.509894,1629.936201
2023-03-25,41230,121431,0x009d4357b163dd89cc4493ddcfb1e6a4ad1f836ea7a5...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,4,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000755e2a226c319c7e5ea9...,NaN,...,Swap,809.320067,0.47,0.0,0,0,0,5.043447e+05,292.774507,1722.638556
2023-03-26,100680,194393,0x159a58f18b9c9695e3bbfc3d3ca8f7e847fcd573e486...,0x80115c708e12edd42e504c1cd52aea96c547c05c,2,15,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000c6c2ee3c9e53053f5de1...,NaN,...,Swap,0.000000,0.0,0.008977,0,0,0,1.234709e+06,700.272831,1763.182135
2023-03-27,150031,269835,0x5300088f6052b52354f803b6404d3f69f60915af6c5f...,0x80115c708e12edd42e504c1cd52aea96c547c05c,1,8,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000b4845049cf818dccd320...,NaN,...,Swap,0.000000,0.0,0.005844,0,0,0,2.217463e+06,1298.43517,1707.796554
2023-03-28,197174,346323,0xfde88eb2747ebf65b4cbb760e19f484a34e1e2130ff3...,0x80115c708e12edd42e504c1cd52aea96c547c05c,2,18,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000eae4aa59051226939fbb...,NaN,...,Swap,1152.048067,0.654935,0.0,0,0,0,4.049821e+06,2296.055663,1763.816466


In [14]:


df_per_day = df_per_day.merge(prices_df[['timestamp', 'Adj Close']], on='timestamp', how='left')
#swap_df['diff'] = swap_df['last_price'] - swap_df['spot_price']
df_per_day.head()

,timestamp,Unnamed: 0,blockNumber,transactionHash,address,transactionIndex,logIndex,topics_0,topics_1,topics_2,...,amount0In,amount1In,amount0Out,amount1Out,price_buy_ETH,last_price,reserve_0,reserve_1,spot_price,Adj Close
0,2023-03-24,1204,43554,0xad594851f951591e5b36f92f35db9b8ef602edb207d2...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,4,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x0000000000000000000000003545744fd133dddfb26b...,...,40.900104,0.025069,0.0,0,0,0,1.387058e+04,8.509894,1629.936201,1752.044800
1,2023-03-25,41230,121431,0x009d4357b163dd89cc4493ddcfb1e6a4ad1f836ea7a5...,0x80115c708e12edd42e504c1cd52aea96c547c05c,0,4,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000755e2a226c319c7e5ea9...,...,809.320067,0.47,0.0,0,0,0,5.043447e+05,292.774507,1722.638556,1743.764771
2,2023-03-26,100680,194393,0x159a58f18b9c9695e3bbfc3d3ca8f7e847fcd573e486...,0x80115c708e12edd42e504c1cd52aea96c547c05c,2,15,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000c6c2ee3c9e53053f5de1...,...,0.000000,0.0,0.008977,0,0,0,1.234709e+06,700.272831,1763.182135,1775.676758
3,2023-03-27,150031,269835,0x5300088f6052b52354f803b6404d3f69f60915af6c5f...,0x80115c708e12edd42e504c1cd52aea96c547c05c,1,8,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000b4845049cf818dccd320...,...,0.000000,0.0,0.005844,0,0,0,2.217463e+06,1298.43517,1707.796554,1715.460815
4,2023-03-28,197174,346323,0xfde88eb2747ebf65b4cbb760e19f484a34e1e2130ff3...,0x80115c708e12edd42e504c1cd52aea96c547c05c,2,18,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0000000000000000000000002da10a1e27bf85cedd8f...,0x000000000000000000000000eae4aa59051226939fbb...,...,1152.048067,0.654935,0.0,0,0,0,4.049821e+06,2296.055663,1763.816466,1772.785645


In [15]:
import plotly.graph_objects as go
#from utils_plot import *
from kaleido import *

colors = {'red': '#ee443a', 'blue': '#42bbf1', 'dark_blue': '#1a4fec',
          'green': '#50be61', 'grey': '#b7b7b7', 'orange': '#f28222', 'purple': '#6e18ee', 'brown': '#a65628', 'pink': '#ef4793',
          'yellow': '#f8c94c', 'black': '#000000', 'white': '#ffffff', 'light_blue': '#a6cee3', 'light_green': '#b2df8a',
          'light_grey': '#999999', 'light_orange': '#fdbf6f', 'light_purple': '#cab2d6', 'light_brown': '#ffff99', 'light_pink': '#1f78b4',
          'light_yellow': '#fb9a99', 'light_black': '#e31a1c', 'light_white': '#33a02c', 'gold': '#ff7f00', 'silver': '#b2df8a'}

layout = go.Layout(
    margin=go.layout.Margin(
        l=10,  # left margin
        r=10,  # right margin
        b=10,  # bottom margin
        t=10  # top margin
    ),
    template='simple_white',
    xaxis=dict(minor_ticks="inside", showgrid=True, griddash='dash',
               minor_griddash="dot"),
    yaxis=dict(minor_ticks="inside", showgrid=True, griddash='dash',
               minor_griddash="dot"),
    font=dict(size=18, family='Clear Sans',
              color='black'),

)

settings = {
    'AddLiquidity': {'color': colors['blue'], 'style': None, 'width': 4.5},
    'RemoveLiquidity': {'color': colors['red'], 'style': 'dash', 'width': 4},
    'RemoveLiquidityOne': {'color': colors['green'], 'style': 'dot', 'width': 3.5},
    'RemoveLiquidityImbalance': {'color': colors['brown'], 'style': 'dashdot', 'width': 3},
}

In [ ]:
fig = go.Figure(layout=layout)

fig.add_trace(go.Scatter(x=swap_df.index,
                            y=swap_df['last_price'].astype(
                                float),
                            mode='lines',
                            name = 'Effective Price'))

fig.add_trace(go.Scatter(x=swap_df.index,
                            y=swap_df['spot_price'].astype(
                                float),
                            mode='lines',
                            name = 'Spot Price'))

fig.add_trace(go.Scatter(x=prices_df['Date'],
                            y=prices_df['Adj Close'].astype(
                                float),
                            mode='lines',
                            name = 'Yahoo Price'))

fig.update_layout(xaxis_title='Date',
                    yaxis_title='Last price', showlegend=True,
                    legend=dict(xanchor='center', x=0.5, y=1.10, orientation='h'),)
#fig.write_image(f"./plots/{contract_name}_{event_name}.pdf")
fig.show()

In [16]:
def calculateMAV(reserves_x, price_AMM, price_CEX):
    return (reserves_x * (price_AMM - price_CEX) / (2 * price_AMM) )
    if (price_AMM > price_CEX) :
        return (reserves_x * (price_AMM - price_CEX) / (2 * price_AMM) )
    else :
        return (reserves_x * (price_CEX - price_AMM) / (2 * price_AMM) )

In [17]:
df_per_day['MAV'] = calculateMAV(df_per_day['reserve_0'], df_per_day['spot_price'], df_per_day['Adj Close'])

In [18]:
df_per_day.head()
df_per_day.columns
MAV =sum(abs(df_per_day['MAV']))
MAV

3145926.9112012316

In [19]:
fig = go.Figure(layout=layout)

fig.add_trace(go.Scatter(x=df_per_day['timestamp'],
                            y=df_per_day['MAV'].astype(
                                float),
                            mode='lines',
                            name = 'ETH-BTC at SyncSwap'))

fig.update_layout(xaxis_title='Date',
                    yaxis_title='Daily MAV [ETH]', showlegend=True,
                    legend=dict(xanchor='center', x=0.5, y=1.10, orientation='h'),)
#fig.write_image(f"./plots/{contract_name}_{event_name}.pdf")
fig.show()

In [20]:
df_per_day['abs_MAV'] = abs(df_per_day['MAV'])
df_per_day['cum_MAV'] = df_per_day['abs_MAV'].cumsum()

In [21]:
fig = go.Figure(layout=layout)

fig.add_trace(go.Scatter(x=df_per_day['timestamp'],
                            y=df_per_day['cum_MAV'].astype(
                                float),
                            mode='lines',
                            name = 'ETH-BTC at SyncSwap'))

fig.update_layout(xaxis_title='Date',
                    yaxis_title='Cumulative MAV [ETH]', showlegend=True,
                    legend=dict(xanchor='center', x=0.5, y=1.10, orientation='h'),)
#fig.write_image(f"./plots/{contract_name}_{event_name}.pdf")
fig.show()

In [22]:
#Phase 1 - reserves and volume
#Phase 2 - fees paid and supply token

df_per_day['volume_ETH'] = df_per_day['amount0In'] + df_per_day['amount0Out'] #this is wrong as it is last, not sum; and assumes price from the pool
df_per_day['reserves_ETH'] = 2 * df_per_day['reserve_0'] #that works only for Uniswap v2 (and its forks)

In [23]:
fig = go.Figure(layout=layout)

fig.add_trace(go.Scatter(x=df_per_day['timestamp'],
                            y=df_per_day['reserves_ETH'].astype(
                                float),
                            mode='lines',
                            name = 'ETH-BTC at SyncSwap'))

fig.update_layout(xaxis_title='Date',
                    yaxis_title='Reserves [ETH]', showlegend=True,
                    legend=dict(xanchor='center', x=0.5, y=1.10, orientation='h'),)
#fig.write_image(f"./plots/{contract_name}_{event_name}.pdf")
fig.show()


In [24]:
df_per_day2 = swap_df.resample('D').agg(dict(amount0In='sum', amount0Out='sum', reserve_0 = 'last'))
df_per_day2['volume_ETH'] = df_per_day2['amount0In'] + df_per_day2['amount0Out'] #this is wrong as it is last, not sum; and assumes price from the pool

df_per_day2.head()


,amount0In,amount0Out,reserve_0,volume_ETH
timestamp,,,,
2023-03-24,2.328426e+04,14.492404,1.387058e+04,23298.755396
2023-03-25,4.835559e+06,2821.041234,5.043447e+05,4838380.32248
2023-03-26,1.344763e+07,7730.086858,1.234709e+06,13455361.9702
2023-03-27,1.124823e+07,6337.890088,2.217463e+06,11254571.542578
2023-03-28,1.239335e+07,7589.119983,4.049821e+06,12400935.864748


In [25]:
fig = go.Figure(layout=layout)

fig.add_trace(go.Scatter(x=df_per_day['timestamp'],
                            y=df_per_day2['volume_ETH'].astype(
                                float),
                            mode='lines',
                            name = 'ETH-BTC at SyncSwap'))

fig.update_layout(xaxis_title='Date',
                    yaxis_title='Trading Volume [ETH]', showlegend=True,
                    legend=dict(xanchor='center', x=0.5, y=1.10, orientation='h'),)
#fig.write_image(f"./plots/{contract_name}_{event_name}.pdf")
fig.show()


In [26]:
def getAnnualReturn(dailyReturn) :
    #return ( dailyReturn * 365) * 100
    return ((dailyReturn + 1)**365 - 1) * 100

df_per_day2['LP_Fee'] = df_per_day2['volume_ETH'] * 0.0008
df_per_day2['reserves_ETH'] = 2 * df_per_day2['reserve_0'] #that works only for Uniswap v2 (and its forks)
df_per_day2['Return'] = df_per_day2['LP_Fee'] / df_per_day2['reserves_ETH']
df_per_day2['annual_Return'] = getAnnualReturn(df_per_day2['Return'])


In [27]:

fig = go.Figure(layout=layout)

fig.add_trace(go.Scatter(x=df_per_day['timestamp'],
                            y=df_per_day2['annual_Return'].astype(
                                float),
                            mode='lines',
                            name = 'ETH-BTC at SyncSwap'))

fig.update_layout(xaxis_title='Date',
                    yaxis_title='Return on Farming [Percent]', showlegend=True,
                    legend=dict(xanchor='center', x=0.5, y=1.10, orientation='h'),)
#fig.write_image(f"./plots/{contract_name}_{event_name}.pdf")
fig.show()



